In [ ]:
import tensorflow as tf
import numpy as np

# Define the dummy dataset
num_users = 1000
num_items = 500
latent_dim = 8
num_factors = 8
mlp_layer_sizes = [64, 32, 16, 8]
regs = [0.01, 0.01, 0.01, 0.01]

user_input = np.random.randint(num_users, size=(10000,))
item_input = np.random.randint(num_items, size=(10000,))
ratings = np.random.randint(2, size=(10000,))





In [ ]:
# Define the NeuMF model
class NeuMF(tf.keras.Model):
    def __init__(self, num_users, num_items, latent_dim, num_factors, mlp_layer_sizes, regs):
        super(NeuMF, self).__init__()

        self.user_embedding = tf.keras.layers.Embedding(num_users, latent_dim)
        self.item_embedding = tf.keras.layers.Embedding(num_items, latent_dim)
        self.flatten = tf.keras.layers.Flatten()

        # Matrix Factorization component
        self.GMF = tf.keras.layers.Dot(axes=1)

        # Multilayer Perceptron component
        self.MLP = tf.keras.Sequential()
        for i in range(len(mlp_layer_sizes)):
            self.MLP.add(tf.keras.layers.Dense(mlp_layer_sizes[i], activation='relu',
                                               kernel_regularizer=tf.keras.regularizers.l2(regs[i])))
        self.concat = tf.keras.layers.Concatenate()
        self.prediction = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        user_input, item_input = inputs
        user_embedded = self.flatten(self.user_embedding(user_input))
        item_embedded = self.flatten(self.item_embedding(item_input))

        # Apply matrix factorization
        GMF_output = self.GMF([user_embedded, item_embedded])

        # Apply multilayer perceptron
        MLP_input = tf.concat([user_embedded, item_embedded], axis=-1)
        MLP_output = self.MLP(MLP_input)

        # Combine the results
        combined_output = self.concat([GMF_output, MLP_output])
        prediction = self.prediction(combined_output)
        
        return prediction




In [ ]:
model = NeuMF(num_users, num_items, latent_dim, num_factors, mlp_layer_sizes, regs)
model.compile(optimizer='adam', loss='binary_crossentropy')
# Fit the model
model.fit([user_input, item_input], ratings, batch_size=256, epochs=10, verbose=1)



Epoch 1/10
40/40 [==============================] - 4s 9ms/step - loss: 1.5296
Epoch 2/10
40/40 [==============================] - 0s 10ms/step - loss: 1.2340
Epoch 3/10
40/40 [==============================] - 0s 11ms/step - loss: 1.0395
Epoch 4/10
40/40 [==============================] - 0s 7ms/step - loss: 0.9133
Epoch 5/10
40/40 [==============================] - 0s 8ms/step - loss: 0.8324
Epoch 6/10
40/40 [==============================] - 0s 11ms/step - loss: 0.7808
Epoch 7/10
40/40 [==============================] - 0s 9ms/step - loss: 0.7483
Epoch 8/10
40/40 [==============================] - 0s 10ms/step - loss: 0.7274
Epoch 9/10
40/40 [==============================] - 0s 10ms/step - loss: 0.7136
Epoch 10/10
40/40 [==============================] - 0s 9ms/step - loss: 0.7034


In [ ]:
# Use the model to make predictions on new data
new_user_input = np.random.randint(num_users, size=(5,))
new_item_input = np.random.randint(num_items, size=(5,))
predictions = model.predict([new_user_input, new_item_input])

print("Predictions: ", predictions)



1/1 [==============================] - 0s 137ms/step
Predictions:  [[0.50010383]
 [0.49431133]
 [0.49494052]
 [0.49084303]
 [0.49379194]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
train_user_input, test_user_input, train_item_input, test_item_input, train_ratings, test_ratings = train_test_split(user_input, item_input, ratings, test_size=0.2)

# Fit the model on the training set
model.fit([train_user_input, train_item_input], train_ratings, batch_size=256, epochs=10, verbose=1)

# Make predictions on the test set
predictions = model.predict([test_user_input, test_item_input])

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(test_ratings, predictions))

print("RMSE: ", rmse)

Epoch 1/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6953
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6876
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6790
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6688
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6569
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6432
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6275
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6102
Epoch 9/10
32/32 [==============================] - 0s 4ms/step - loss: 0.5913
Epoch 10/10
63/63 [==============================] - 0s 3ms/step
RMSE:  0.47471766967871126
